# memo

In [1]:
import numpy as np
import pandas as pd

## pandas

In [2]:
df = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(5)),dtype='float32'),
                    'D' : np.array([3] * 5,dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train", "test"]),
                    'F' : 'foo',
                    'G' : ['1000', np.nan, '123', '1234', '']})

In [3]:
df

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,NaN
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234
4,1.0,2013-01-02,1.0,3,test,foo,


## NaN処理

In [4]:
# NaNであるか
df.isna()

,A,B,C,D,E,F,G
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False


In [5]:
# 全項目NaN化
df[df.isna()]

,A,B,C,D,E,F,G
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [6]:
# blankがNaN
df[~df.isna()]

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,NaN
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234
4,1.0,2013-01-02,1.0,3,test,foo,


## replace NaN

In [7]:
# EX) df.replace([-1, 999, 1000], np.nan)
df = df.replace('', np.nan)
df

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,NaN
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234
4,1.0,2013-01-02,1.0,3,test,foo,NaN


## NaNを0,000に変換

In [8]:
df[df['G'].isnull()].replace(np.nan, '0,000')

,A,B,C,D,E,F,G
1,1.0,2013-01-02,1.0,3,train,foo,"0,000"
4,1.0,2013-01-02,1.0,3,test,foo,"0,000"


- 価格区切り

In [9]:
"{:,}".format(12345678)

'12,345,678'

In [10]:
i = 12345678
f"{i:,}"

'12,345,678'

- strやから無理

In [11]:
str_ = '12345678'
f"{str_:}"

'12345678'

- round 組み込み関数

In [12]:
round(12.345678, 3)

12.346

## 「記号-数字」の組み合わせをソート(改変2020/09/12)

In [13]:
# sample date
df = pd.DataFrame({ 'A' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'B' : ['A-10', 'B-3', 'A-1', 'C-2', np.nan,'A-2', 'B-2', 'B-1', 'C-1', np.nan],})

In [14]:
df

,A,B
0,1,A-10
1,2,B-3
2,3,A-1
3,4,C-2
4,5,NaN
5,6,A-2
6,7,B-2
7,8,B-1
8,9,C-1
9,10,NaN


In [15]:
def demarcate_hyphen(df, before_col="B", sort_parent='sort_parent', sort_child='sort_child'):
    """ 2020/09/10
    
    アルファベット、数字の組み合わせがある値をソートする。
    そのままソートすると数字が文字列の為上手くソートできない
    ハイフンで切り分けて、数字はnp.intにキャストした
    
    ExSample:
        before  after
        --------------
        A-10    A-1      
        B-3     A-10
        A-1     B-3
        C-2     C-2
        NaN         <- NaNは削除
                
    params:
    	df(DataFrame)	 : 処理してるデータ
        before_col(str)  : 処理対象(column name)
        sort_parent(str) : ソートする素(column name)
        sort_child(str)  : ソートする付属(column name)
        
    return:
        ソートした結果返す
    """
    
    # NaN(float)を除外. floatでソートできない為
    filter_ = df[~df[before_col].isna()]
    
    df[sort_parent] = filter_.apply(
        lambda row: f"{row[before_col]}".split('-')[0] if '-' in row[before_col] else row[before_col], axis=1)
 
    df[sort_child] = filter_.apply(
        lambda row: f"{row[before_col]}".split('-')[1] if '-' in row[before_col] else 0, axis=1).astype(np.int)
    
    df = df.sort_values([sort_parent, sort_child]).fillna('')
    
    return df

In [16]:
df = demarcate_hyphen(df, before_col="B", sort_parent='sort_parent', sort_child='sort_child')
df

,A,B,sort_parent,sort_child
2,3,A-1,A,1
5,6,A-2,A,2
0,1,A-10,A,10
7,8,B-1,B,1
6,7,B-2,B,2
1,2,B-3,B,3
8,9,C-1,C,1
3,4,C-2,C,2
4,5,,,
9,10,,,


### 最終表示 sortできた

In [17]:
df = df[['A', 'B']]
df

,A,B
2,3,A-1
5,6,A-2
0,1,A-10
7,8,B-1
6,7,B-2
1,2,B-3
8,9,C-1
3,4,C-2
4,5,
9,10,


In [18]:
df.to_csv('test.csv', index=False)

In [19]:
pos_inf = float("inf")

In [20]:
pos_inf

inf

In [21]:
pos_inf = float("-inf")

In [22]:
type(pos_inf)

float

In [23]:
# infを呼び出す
x = np.inf

In [24]:
x

inf

In [25]:
type(x)

float

## 「記号-数字」の組み合わせをソート(改変2020/09/12)

In [26]:
# sample date
df = pd.DataFrame({ 'A' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'B' : ['A-10', 'B-3', 'A-1', 'C-2', np.nan,'A-2', 'B-2', 'B-1', 'C-1', np.nan],})

In [27]:
df

,A,B
0,1,A-10
1,2,B-3
2,3,A-1
3,4,C-2
4,5,NaN
5,6,A-2
6,7,B-2
7,8,B-1
8,9,C-1
9,10,NaN


In [28]:
def demarcate_hyphen(df, before_col='B', sort_col='sort_col'):
    """
    ソートできるようにハイフンで切り分ける
    数字が文字列で上手くsortされない為
    
    params:
    	df(DataFrame)	 : 処理してるデータ
        before_col(str)  : ソート素対象(column name)
        sort_col(str)    : ソート用(column name)
        
    return:
        ソート用に切り分けたcolumnを返す
    """
    df[sort_col] = pd.Series(df[before_col].str.split('-'))
    
    return df

In [29]:
def _carve_element(sort_col='sort_col'):
    """ listにしたカラムを要素で切り分けする
    
    params:
        sort_col(str) : ソート用(column name)
        
    return:
        要素で切り分けた結果を返す
    """
    
    return pd.Series([sort_col[0], np.int(sort_col[1])])

In [30]:
def create_sort_col(df, sort_col='sort_col', sort_parent="symbol", sort_child='num'):
    """
    sortするカラムを作成する
    
    params:
    	df(DataFrame)	 : 処理してるデータ
        sort_col(str)    : ソート素(column name)
        sort_parent(str) : ソートする基準(column name)
        sort_child(str)  : ソートする付属(column name)
    
    return:
        sort用に作成したカラムを返す
    """
    
    # NaN除外.floatでnp.intにキャストできない
    filter_ = df[~df[sort_col].isna()]

    df[[sort_parent, sort_child]] = filter_[sort_col].apply(_carve_element)
    
    return df

In [31]:
def sort_symbol(df, sort_parent="symbol", sort_child='num'):
    """ 2020/09/10
    
    アルファベット、数字の組み合わせがある値をソートする。
    
    ExSample:
        before  after
        --------------
        A-10    A-1      
        B-3     A-10
        A-1     B-3
        C-2     C-2
        NaN         <- NaNは削除
                
    params:
    	df(DataFrame)	 : 処理してるデータ
        sort_parent(str) : ソートする基準(column name)
        sort_child(str)  : ソートする付属(column name)
        
    return:
        ソートした結果返す
    """
    df = df.sort_values([sort_parent, sort_child]).fillna('')

    return df

In [32]:
# df = demarcate_hyphen(df, before_col="B", sort_col='sort_col')
# sort_symbol(df, sort_parent="symbol", sort_child='num')

## 実行処理

In [33]:
df = demarcate_hyphen(df, before_col='B', sort_col='sort_col')
df = create_sort_col(df, sort_col='sort_col', sort_parent="symbol", sort_child='num')
df = sort_symbol(df, sort_parent="symbol", sort_child='num')
df

,A,B,sort_col,symbol,num
2,3,A-1,"[A, 1]",A,1
5,6,A-2,"[A, 2]",A,2
0,1,A-10,"[A, 10]",A,10
7,8,B-1,"[B, 1]",B,1
6,7,B-2,"[B, 2]",B,2
1,2,B-3,"[B, 3]",B,3
8,9,C-1,"[C, 1]",C,1
3,4,C-2,"[C, 2]",C,2
4,5,,,,
9,10,,,,


### 最終表示 sortできた

In [34]:
df[['A', 'B']]

,A,B
2,3,A-1
5,6,A-2
0,1,A-10
7,8,B-1
6,7,B-2
1,2,B-3
8,9,C-1
3,4,C-2
4,5,
9,10,


# 📓MEMO

In [35]:
def some_function1(x):
    return pd.Series([x*2, x/2])

In [36]:
df[['new_column1', 'new_columns2']] = df['A'].apply(some_function1)

In [37]:
df

,A,B,sort_col,symbol,num,new_column1,new_columns2
2,3,A-1,"[A, 1]",A,1,6.0,1.5
5,6,A-2,"[A, 2]",A,2,12.0,3.0
0,1,A-10,"[A, 10]",A,10,2.0,0.5
7,8,B-1,"[B, 1]",B,1,16.0,4.0
6,7,B-2,"[B, 2]",B,2,14.0,3.5
1,2,B-3,"[B, 3]",B,3,4.0,1.0
8,9,C-1,"[C, 1]",C,1,18.0,4.5
3,4,C-2,"[C, 2]",C,2,8.0,2.0
4,5,,,,,10.0,2.5
9,10,,,,,20.0,5.0


In [38]:
df.sort_values(['B']).fillna('')

,A,B,sort_col,symbol,num,new_column1,new_columns2
4,5,,,,,10.0,2.5
9,10,,,,,20.0,5.0
2,3,A-1,"[A, 1]",A,1,6.0,1.5
0,1,A-10,"[A, 10]",A,10,2.0,0.5
5,6,A-2,"[A, 2]",A,2,12.0,3.0
7,8,B-1,"[B, 1]",B,1,16.0,4.0
6,7,B-2,"[B, 2]",B,2,14.0,3.5
1,2,B-3,"[B, 3]",B,3,4.0,1.0
8,9,C-1,"[C, 1]",C,1,18.0,4.5
3,4,C-2,"[C, 2]",C,2,8.0,2.0


In [39]:
# df[sort_parent] = filter_.apply(
#     lambda row: f"{row[before_col]}".split('-')[0] if '-' in row[before_col] else row[before_col], axis=1)

# df[sort_child] = filter_.apply(
#     lambda row: f"{row[before_col]}".split('-')[1] if '-' in row[before_col] else 0, axis=1).astype(np.int)

# df = df.sort_values([sort_parent, sort_child]).fillna('')

### sampleリストの動き確認

In [40]:
l = ['Banana', 'Alice', 'Apple', 'Bob', 'xxx']
l_order = ['Alice', 'Bob', 'Apple', 'Banana']

In [41]:
sorted(l, key=lambda x: l_order.index(x) if x in l_order else -1)

['xxx', 'Alice', 'Bob', 'Apple', 'Banana']

In [42]:
l = ['Banana', 'Alice', 'Apple', 'Bob', 'xxx']
l_order = ['Alice', 'Bob', 'Apple', 'Banana']

In [43]:
sorted(l, key=lambda x: l_order.index(x) if x in l_order else float('inf'))

['Alice', 'Bob', 'Apple', 'Banana', 'xxx']